In [3]:
import numpy as np
import pandas as pd


df = pd.read_csv("transactions_train.csv", dtype={"article_id": str})
print(df.shape)
df.head()

(31788324, 5)


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2


In [4]:
# t_dat 열을 날짜 형식으로 변환하고 데이터셋의 최신 날짜를 확인함
df["t_dat"] = pd.to_datetime(df["t_dat"])
df["t_dat"].max()

Timestamp('2020-09-22 00:00:00')

In [5]:
# 최근 활성화된 고객들을 필터링함
active_articles = df.groupby("article_id")["t_dat"].max().reset_index()
active_articles = active_articles[active_articles["t_dat"] >= "2019-09-01"].reset_index()
active_articles.shape

(72581, 3)

In [6]:
df = df[df["article_id"].isin(active_articles["article_id"])].reset_index(drop=True)
df.shape

(29634404, 5)

In [7]:
df["week"] = (df["t_dat"].max() - df["t_dat"]).dt.days // 7
df["week"].value_counts()

week
65     620104
13     549443
42     518403
12     517428
64     508664
        ...  
93     174190
102    164298
104    163143
97     162580
94     152807
Name: count, Length: 105, dtype: int64

In [11]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 6.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 6.4 MB/s eta 0:00:0000:0100:01


In [12]:
# 고객ID를 정수형 코드로 변환해서 모델이 처리하기 쉽게 함
from sklearn.preprocessing import LabelEncoder


article_ids = np.concatenate([["placeholder"], np.unique(df["article_id"].values)])

le_article = LabelEncoder()
le_article.fit(article_ids)
df["article_id"] = le_article.transform(df["article_id"])

In [13]:
# 학습, 검증을 위한 데이터셋 생성
WEEK_HIST_MAX = 5

# 목적: 지정된 week에 대한 훈련 데이터를 생성하고 해당 주차를 예측하는 데 사용될 고객의 과거 구매이력을 포함하는 데이터셋 생성
def create_dataset(df, week):
    # 지정된 주차 'week' 이후의 데이터를 수집하여 최대 5주간의 구매 이력을 추출함
    hist_df = df[(df["week"] > week) & (df["week"] <= week + WEEK_HIST_MAX)]
    # 고객별로 그룹화하여 'article_id'와 'week' 정보를 리스트 형태로 저장
    hist_df = hist_df.groupby("customer_id").agg({"article_id": list, "week": list}).reset_index()
    hist_df.rename(columns={"week": 'week_history'}, inplace=True)

    # 지정된 'week'에서의 고객별 구매 내역을 타겟으로 설정함
    target_df = df[df["week"] == week]
    # 고객별로 그룹화하여 'article_id'를 리스트로 저장. 해당 주차에 고객이 구매한 제품 목록임
    target_df = target_df.groupby("customer_id").agg({"article_id": list}).reset_index()
    target_df.rename(columns={"article_id": "target"}, inplace=True)
    target_df["week"] = week
    
    # 과거 데이터와 타겟 데이터를 'customer_id'를 기준으로 조인함. 
    return target_df.merge(hist_df, on="customer_id", how="left")

# 0주차에 대한 데이터셋을 생성하여 모델 평가에 사용함
val_weeks = [0]
# 1~4주차에 대한 데이터셋 생성하여 모델 학습에 사용함
train_weeks = [1, 2, 3, 4]

# create_dataset 함수 사용하여 데이터들을 병합하여 최종적인 데이터셋 형성
val_df = pd.concat([create_dataset(df, w) for w in val_weeks]).reset_index(drop=True)
train_df = pd.concat([create_dataset(df, w) for w in train_weeks]).reset_index(drop=True)
train_df.shape, val_df.shape

((300129, 5), (68984, 5))

In [15]:
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 986.0 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 4.3 MB/s eta 0:00:00


In [16]:
# torch 사용하여 사용자 정의 데이터셋과 데이터 로더를 설정하고 모델 훈련시킴

from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm

class HMDataset(Dataset):
    def __init__(self, df, seq_len, is_test=False):
        self.df = df.reset_index(drop=True)
        self.seq_len = seq_len
        self.is_test = is_test

    def __len__(self):
        return self.df.shape[0]

    # 데이터 포매팅 및 전처리 로직 구현
    def __getitem__(self, index):
        row = self.df.iloc[index]

        if self.is_test:
            # 빈 타겟 텐서를 생성함
            target = torch.zeros(2).float()
        else:
            # 빈 타겟 텐서를 생성하고 row.target에 포함된 각 아이템에 대해 해당 인덱스를 1로 설정함(원핫인코딩방식)
            target = torch.zeros(len(article_ids)).float()
            for t in row.target:
                target[t] = 1.0

        # article_hist(고객의 과거 구매 ID)와 week_hist(해당 구매가 발생한 시간의 상대적 차이) 텐서 생성.
        article_hist = torch.zeros(self.seq_len).long()
        week_hist = torch.ones(self.seq_len).float()

        if isinstance(row.article_id, list):
            if len(row.article_id) >= self.seq_len:
                # 가장 최근의 seq_len개의 article_id와 week_history 추출
                article_hist = torch.LongTensor(row.article_id[-self.seq_len:])
                week_hist = (torch.LongTensor(row.week_history[-self.seq_len:]) - row.week)/WEEK_HIST_MAX/2
            else:
                # 필요한 길이만큼만 데이터를 할당하고 나머지는 0 또는 1로 채움 (시퀀스 데이터 처리에서 일반적으로 사용되는 방법임)
                article_hist[-len(row.article_id):] = torch.LongTensor(row.article_id)
                week_hist[-len(row.article_id):] = (torch.LongTensor(row.week_history) - row.week)/WEEK_HIST_MAX/2

        return article_hist, week_hist, target

# val_df 데이터 프레임을 사용하여 인스턴스 생성하고 시퀀스 길이를 64로 설정한 뒤 인덱스 1 데이터 불러와서 확인함
HMDataset(val_df, 64)[1]

(tensor([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0, 70912, 69932, 70149, 14648,
         11949, 66254, 66254, 67809]),
 tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
 

In [17]:
# 목적: 주어진 epoch에 따라 optimizer의 학습률을 조정함
def adjust_lr(optimizer, epoch):
    if epoch < 1:
        # 학습 초기에 매우 낮은 학습률로 시작함
        lr = 5e-5
    elif epoch < 6:
        # 학습률을 증가시켜 빠른 학습 추진함
        lr = 1e-3
    elif epoch < 9:
        # 학습률을 감소시켜 미세하게 조정함
        lr = 1e-4
    else:
        # 매우 낮은 학습률로 마무리 단계에서 세밀하게 조정함
        lr = 1e-5

    # 설정된 lr을 optimizer의 모든 매개변수 그룹에 적용함. 
    for p in optimizer.param_groups:
        p['lr'] = lr
    return lr

# 목적: 주어진 신경망 모델(net)에 대해 Adam 최적화 도구를 설정하고 반환함.
# Adam최적화: 자동으로 학습률을 조정하면서 모멘텀을 사용하여 최적화를 수행하는 알고리즘
def get_optimizer(net):
    # requires_grad가 True인 모델 파라미터만 최적화 대상으로 선택함(학습 가능한 파라미터만 최적화하겠다는 뜻)
    # betas = (0.9, 0.999): Adam 최적화의 두 모멘텀 파라미터 beta1과 beta2
    # eps = 1e-08: 수치안정성을 위한 작은 상수
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=3e-4, betas=(0.9, 0.999),
                                 eps=1e-08)
    return optimizer

### 신경망 아키텍쳐 HMModel를 정의하고 초기화 하고 실행하는 과정임

In [18]:
import torch.nn as nn
import torch.nn.functional as F

class HMModel(nn.Module):

    # 초기화 함수
    def __init__(self, article_shape):
        super(HMModel, self).__init__()
        # article_emb: article_id를 벡터로 변환하는데 사용됨. 각 article_id를 고차원 공간에서의 밀집 벡터로 표현하여 아티클 간의 관계를 학습하는데 도움을 줌
        self.article_emb = nn.Embedding(article_shape[0], embedding_dim=article_shape[1])
        # article_likelihood: 각 article의 구매 가능성을 나타내는 학습 가능한 파라미터로, 초기값은 모두 0임
        self.article_likelihood = nn.Parameter(torch.zeros(article_shape[0]), requires_grad=True)
        # top: 컨볼루션 레이어. 입력 특성을 변환하기 위한 일련의 1D 컨볼루션 레이어와 LeakyReLU 활성화 함수를 포함함. 최종 출력을 생성하기 전에 특성들을 더 추출하고 변환함
        self.top = nn.Sequential(nn.Conv1d(3, 32, kernel_size=1), nn.LeakyReLU(),
                                 nn.Conv1d(32, 8, kernel_size=1), nn.LeakyReLU(),
                                 nn.Conv1d(8, 1, kernel_size=1))
    
    # 순전파 함수
    # 단계: 임베딩 레이어 -> 유사성 계산 -> 통계적 처리 -> 특성 병합과 컨볼루션 처리
    def forward(self, inputs):

        # 1. 입력데이터 분리
        article_hist, week_hist = inputs[0], inputs[1]

        # 2. 임베딩 레이어를 통한 처리
        # article_hist가 임베딩 레이어 self.article_emb를 통해 각 article_id를 해당하는 벡터로 변환함
        x = self.article_emb(article_hist)
        # 변환된 벡터는 F.normalize 함수를 통해 정규화됨
        # -> 벡터의 길이를 1로 맞춰서 다음 계산에서 수치적 안정성을 높이는 역할임
        x = F.normalize(x, dim=2)

        # 3. 임베딩 가중치와의 내적
        # 정규화된 임베딩벡터 x와 임베딩레이어의 가중치 self.article_emb.weight의 전치와 내적을 계산함.
        # -> 각 벡터가 다른 벡터들과 얼마나 유사한지 평가하는 점수를 생성함
        x = x@F.normalize(self.article_emb.weight).T

        # 4. 최대 유사성 추출
        # 각 샘플에 대해 가장 높은 유사성 점수와 해당 인덱스를 추출함
        x, indices = x.max(axis=1)
        # clamp 함수를 사용해 점수를 [0.001, 0.999] 범위로 제한함
        # -> 로그 연산시 발생할 수 있는 수치 문제를 방지하는 역할임
        x = x.clamp(1e-3, 0.999)
        # 로지스틱 시그모이드 함수의 역함수를 적용하여 로짓변환함
        x = -torch.log(1/x - 1)

        # 5. week 정보 집계
        # week_hist에서 incides에 해당하는 최대 유사성의 week 정보를 추출함
        max_week = week_hist.unsqueeze(2).repeat(1, 1, x.shape[-1]).gather(1, indices.unsqueeze(1).repeat(1, week_hist.shape[1], 1))
        # 이를 평균내어 해당 샘플의 평균 구매 시점을 계산함
        max_week = max_week.mean(axis=1).unsqueeze(1)

        # 6. 특성 병합 및 최정 컨볼루션 레이어 처리
        # torch.cat를 사용해 특성들을 병합함
        x = torch.cat([x.unsqueeze(1), max_week,
                       self.article_likelihood[None, None, :].repeat(x.shape[0], 1, 1)], axis=1)
        # 병합된 특성을 컨볼루션레이어 시퀀스 self.top에 통과시켜서 최종 출력을 생성함
        x = self.top(x).squeeze(1)
        return x

# HMModel 인스턴스를 생성하고 article_id의 수와 임베딩 차원을 설정함
model = HMModel((len(le_article.classes_), 512))
# 모델을 GPU로 이동시킴
model = model.cuda()

### 신경망 모델을 검증하는 과정 설명함

In [62]:
import sys

# 목적: 예측과 실제를 비교해서 주어진 k값에 대해 평균정밀도(MAP)를 계산함
# topk_preds: 모델이 예측한 상위 k개의 결과 인덱스 리스트
def calc_map(topk_preds, target_array, k=12):
    # 각 예측에 대한 정밀도를 저장할 리스트
    metric = []
    # true positives, false positives
    tp, fp = 0, 0

    for pred in topk_preds:
        # 인덱스에 해당하는 타겟 값이 참(1)이면(=실제로 양성이면)
        if target_array[pred]:
            # tp 값 1만큼 증가시킴
            tp += 1
            # metric 리스트에 현재까지의 정밀도를 추가함
            metric.append(tp/(tp + fp))
        else:
            # 아니면 fp값 1만큼 증가시킴
            fp += 1
    # 계산된 모든 정밀도 값의 합을 k와 실제 양성 샘플수의 최솟값으로 나눔
    # min을 한 이유는 실제 양성 샘플의 수가 k보다 적을 수 있기 때문
    return np.sum(metric) / min(k, target_array.sum())

# 배치로부터 입력데이터와 타겟데이터를 분리해서 GPU로 이동시킴
def read_data(data):
    # 입력 데이터 튜플과 타겟데이터를 반환함
    return tuple(d.cuda() for d in data[:-1]), data[-1].cuda()

# 주어진 모델을 사용하여 검증 데이터셋에서 성능을 평가하고 평균정밀도(MAP)을 계산하는 함수
def validate(model, val_loader, k=12):
    # 모델을 평가모드로 설정함
    model.eval()

    tbar = tqdm(val_loader, file=sys.stdout)

    maps = []
    # torch.no_grad(): 자동미분기능을 비활성화해서 메모리 사용을 줄이고 계산 속도를 향상시킴
    with torch.no_grad():
        # 데이터 로더에서 배치 데이터를 순차적으로 가져옴
        for idx, data in enumerate(tbar):
            # 각 배치 데이터를 읽고 GPU로 이동시킴
            inputs, target = read_data(data)
            # 모델에 입력데이터를 전달하여 로짓(에측 결과의 원시점수)를 계산함
            logits = model(inputs)
            # 로짓 중 확신도? 상위 k개를 선택함 (확신도: 모델이 예측한 각 분류결과에 대해서 얼마나 자신있는지..? 정답일 가능성)
            _, indices = torch.topk(logits, k, dim=1)
            # GPU에서 계산된 결과를 CPU로 옮기고 넘파이 배열로 변환함
            indices = indices.detach().cpu().numpy()
            target = target.detach().cpu().numpy()
            # 각 예측에 대해 MAP를 계산하고 리스트에 추가함
            for i in range(indices.shape[0]):
                maps.append(calc_map(indices[i], target[i]))

    # 계산된 모든 MAP 값의 평균을 반환함. 평균값은 모델의 전반적인 성능을 나타냄
    return np.mean(maps)

# 모델이 처리할 각 입력데이터의 길이 또는 시퀀스의 요소 수 지정함
SEQ_LEN = 16
# 배치사이즈
BS = 32
# DataLoader에서 사용할 작업자(Worker)의 수. 데이터로딩을 위해 동시에 실행할 프로세스의 수
NW = 2

# 모델학습과 검증에 적합한 데이터 형태로 변환함
val_dataset = HMDataset(val_df, SEQ_LEN)
# Dataset에서 제공하는 데이터를 모델이 사용할 수 있도록 배치 단위로 묶어주는 역할을 함
val_loader = DataLoader(val_dataset,
                        batch_size=BS,
                        shuffle=False, # 데이터를 섞지 않고 순차적으로 로드함
                        num_workers=NW,
                        pin_memory=False, # DataLoader가 텐서를 CUDA 고정 메모리에 올리지 않도록 함. 데이터를 GPU로 더 빠르게 전송할 수 있게 해줌
                        drop_last=False # 마지막 배치가 설정된 배치사이즈보다 작을 경우 이를 버리지 않고 사용함
                        )

### 손실함수(dice_loss) 구현하고 신경망모델 훈련하고 검증함

In [63]:
# 딥러닝에서 자주 사용되는 손실함수중 하나인 Dice Loss를 구현한거임. 주로 두 샘플 간의 유사성을 측정하는데 사용되고 특히 불균형한 데이터셋에 유영함.
# 예측결과와 실제값 사이의 유사도를 계산하여 두 데이터의 겹치는 부분이 클수록 손실을 줄임
def dice_loss(y_pred, y_true):
    # 모델의 원시출력
    y_pred = y_pred.sigmoid()
    intersect = (y_true*y_pred).sum(axis=1)

    return 1 - (intersect/(intersect + y_true.sum(axis=1) + y_pred.sum(axis=1))).mean()


def train(model, train_loader, val_loader, epochs):
    np.random.seed(SEED)

    optimizer = get_optimizer(model)
    scaler = torch.cuda.amp.GradScaler()

    criterion = torch.nn.BCEWithLogitsLoss()

    for e in range(epochs):
        model.train()
        tbar = tqdm(train_loader, file=sys.stdout)

        lr = adjust_lr(optimizer, e)

        loss_list = []

        for idx, data in enumerate(tbar):
            inputs, target = read_data(data)

            optimizer.zero_grad()

            with torch.cuda.amp.autocast():
                logits = model(inputs)
                loss = criterion(logits, target) + dice_loss(logits, target)


            #loss.backward()
            scaler.scale(loss).backward()
            #optimizer.step()
            scaler.step(optimizer)
            scaler.update()

            loss_list.append(loss.detach().cpu().item())

            avg_loss = np.round(100*np.mean(loss_list), 4)

            tbar.set_description(f"Epoch {e+1} Loss: {avg_loss} lr: {lr}")

        val_map = validate(model, val_loader)

        log_text = f"Epoch {e+1}\nTrain Loss: {avg_loss}\nValidation MAP: {val_map}\n"

        print(log_text)

        #logfile = open(f"models/{MODEL_NAME}_{SEED}.txt", 'a')
        #logfile.write(log_text)
        #logfile.close()
    return model


MODEL_NAME = "exp001"
SEED = 0

train_dataset = HMDataset(train_df, SEQ_LEN)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=NW,
                          pin_memory=True, drop_last=True)

model = train(model, train_loader, val_loader, epochs=10)

100%|███████████████████████████████████████████████████████████████████████████████| 2156/2156 [00:46<00:00, 46.11it/s]
Epoch 1
Train Loss: 99.4574
Validation MAP: 0.023463344107770078

100%|███████████████████████████████████████████████████████████████████████████████| 2156/2156 [00:46<00:00, 46.82it/s]
Epoch 2
Train Loss: 99.3079
Validation MAP: 0.023983598739893707

100%|███████████████████████████████████████████████████████████████████████████████| 2156/2156 [00:48<00:00, 44.64it/s]
Epoch 3
Train Loss: 99.2424
Validation MAP: 0.02311642030903419

100%|███████████████████████████████████████████████████████████████████████████████| 2156/2156 [00:46<00:00, 46.24it/s]
Epoch 4
Train Loss: 99.2212
Validation MAP: 0.023058467315314645

100%|███████████████████████████████████████████████████████████████████████████████| 2156/2156 [00:47<00:00, 45.09it/s]
Epoch 5
Train Loss: 99.2004
Validation MAP: 0.023266210282059687

100%|█████████████████████████████████████████████████████████████

In [64]:
combined_df = pd.concat([train_df[train_df["week"] < 4], val_df])
train_dataset = HMDataset(combined_df, SEQ_LEN)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=NW,
                          pin_memory=False, drop_last=True)
model = train(model, train_loader, val_loader, epochs=10)

100%|███████████████████████████████████████████████████████████████████████████████| 2156/2156 [00:51<00:00, 41.64it/s]
Epoch 1
Train Loss: 99.1118
Validation MAP: 0.02281379879790295

100%|███████████████████████████████████████████████████████████████████████████████| 2156/2156 [00:51<00:00, 41.53it/s]
Epoch 2
Train Loss: 99.1198
Validation MAP: 0.02518466185549604

100%|███████████████████████████████████████████████████████████████████████████████| 2156/2156 [00:51<00:00, 41.53it/s]
Epoch 3
Train Loss: 99.0855
Validation MAP: 0.025543896871232236

100%|███████████████████████████████████████████████████████████████████████████████| 2156/2156 [00:52<00:00, 41.41it/s]
Epoch 4
Train Loss: 99.0593
Validation MAP: 0.02601509394211474

100%|███████████████████████████████████████████████████████████████████████████████| 2156/2156 [00:49<00:00, 43.43it/s]
Epoch 5
Train Loss: 99.0456
Validation MAP: 0.026751584260624288

100%|███████████████████████████████████████████████████████████████

In [65]:
test_df = pd.read_csv('sample_submission.csv').drop("prediction", axis=1)
print(test_df.shape)
test_df.head()

(1371980, 1)


,customer_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...


In [66]:
def create_test_dataset(test_df):
    week = -1
    test_df["week"] = week

    hist_df = df[(df["week"] > week) & (df["week"] <= week + WEEK_HIST_MAX)]
    hist_df = hist_df.groupby("customer_id").agg({"article_id": list, "week": list}).reset_index()
    hist_df.rename(columns={"week": 'week_history'}, inplace=True)


    return test_df.merge(hist_df, on="customer_id", how="left")

test_df = create_test_dataset(test_df)
test_df.head()

,customer_id,week,article_id,week_history
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,-1,[7154],[2]
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,-1,NaN,NaN
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,-1,[46435],[1]
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,-1,NaN,NaN
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,-1,NaN,NaN


In [67]:
test_df["article_id"].isnull().mean()

0.8008965145264508

In [68]:
test_ds = HMDataset(test_df, SEQ_LEN, is_test=True)
test_loader = DataLoader(test_ds, batch_size=BS, shuffle=False, num_workers=NW,
                          pin_memory=False, drop_last=False)


def inference(model, loader, k=12):
    model.eval()

    tbar = tqdm(loader, file=sys.stdout)

    preds = []

    with torch.no_grad():
        for idx, data in enumerate(tbar):
            inputs, target = read_data(data)

            logits = model(inputs)

            _, indices = torch.topk(logits, k, dim=1)

            indices = indices.detach().cpu().numpy()
            target = target.detach().cpu().numpy()

            for i in range(indices.shape[0]):
                preds.append(" ".join(list(le_article.inverse_transform(indices[i]))))


    return preds

# 테스트 데이터셋 예측 생성
test_df["prediction"] = inference(model, test_loader)

100%|█████████████████████████████████████████████████████████████████████████████| 42875/42875 [26:59<00:00, 26.48it/s]


In [69]:
# 결과 파일 생성 및 저장
test_df.to_csv("submission.csv", index=False, columns=["customer_id", "prediction"])